In [ ]:
import sagemaker
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()
role = "arn:aws:iam::941656036254:role/service-role/AmazonSageMaker-ExecutionRole-20210904T193230"

bucket = sagemaker_session.default_bucket()
prefix = 'sagemaker/pytorch-distribution-options'
print('Bucket:\n{}'.format(bucket))

In [ ]:
# preparing dataset
! wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
! unzip hymenoptera_data.zip

In [ ]:
data_url = sagemaker_session.upload_data(path="./hymenoptera_data", key_prefix="hymenoptera_data")

In [ ]:
data_url

# PyTorch DDP

In [ ]:
from sagemaker.pytorch import PyTorch

#ps_instance_type = 'ml.p3.2xlarge'
ps_instance_type = 'ml.p3.2xlarge'
ps_instance_count = 2

#distribution = {'parameter_server': {
#                    'enabled': True}
#                }
hyperparameters = {
  'train-script': 'train_ddp.py',
  'epochs': 4,
  #'batch-size-per-device' : 16,
  #'steps-per-epoch': 100
  }

estimator_ms = PyTorch(
                       source_dir='2_sources',
                       entry_point='launcher.py', 
                       role=role,
                       framework_version='1.9',
                       py_version='py38',
                       disable_profiler=True,
                       debugger_hook_config=False,
                       hyperparameters=hyperparameters,
                       instance_count=ps_instance_count, 
                       instance_type=ps_instance_type,
                       )

estimator_ms.fit(inputs={"train":f"{data_url}/train", "val":f"{data_url}/val"})